## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Yulara,AU,-25.2406,130.9889,67.84,56,0,2.30,clear sky
1,1,Castro,BR,-24.7911,-50.0119,75.07,65,37,5.53,scattered clouds
2,2,Sitka,US,57.0531,-135.3300,37.29,52,100,8.05,overcast clouds
3,3,Cape Town,ZA,-33.9258,18.4232,68.09,61,0,8.05,clear sky
4,4,Kapaa,US,22.0752,-159.3190,77.18,71,0,1.99,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Castro,BR,-24.7911,-50.0119,75.07,65,37,5.53,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,77.18,71,0,1.99,clear sky
5,5,Rikitea,PF,-23.1203,-134.9692,79.21,72,89,18.81,overcast clouds
7,7,Itarema,BR,-2.9248,-39.9167,83.48,64,89,10.38,overcast clouds
10,10,Kismayo,SO,-0.3582,42.5454,77.22,80,4,17.29,clear sky
11,11,Cidreira,BR,-30.1811,-50.2056,76.23,74,1,16.46,clear sky
14,14,Vaini,TO,-21.2000,-175.2000,81.50,80,81,21.97,broken clouds
15,15,Cotonou,BJ,6.3654,2.4183,84.18,84,20,11.50,few clouds
17,17,Lorengau,PG,-2.0226,147.2712,80.08,82,100,6.35,overcast clouds
18,18,Tres Lagoas,BR,-20.7511,-51.6783,85.21,54,96,6.76,overcast clouds


In [9]:
# get a count of preferred cities based on input criteria
preferred_cities_df.count()

City_ID                219
City                   219
Country                218
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Weather Description    219
dtype: int64

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
try:
    clean_city_df=preferred_cities_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_city_df=preferred_cities_df

In [12]:
# get a count of preferred cities based on input criteria
clean_city_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Castro,BR,75.07,scattered clouds,-24.7911,-50.0119,
4,Kapaa,US,77.18,clear sky,22.0752,-159.3190,
5,Rikitea,PF,79.21,overcast clouds,-23.1203,-134.9692,
7,Itarema,BR,83.48,overcast clouds,-2.9248,-39.9167,
10,Kismayo,SO,77.22,clear sky,-0.3582,42.5454,
11,Cidreira,BR,76.23,clear sky,-30.1811,-50.2056,
14,Vaini,TO,81.50,broken clouds,-21.2000,-175.2000,
15,Cotonou,BJ,84.18,few clouds,6.3654,2.4183,
17,Lorengau,PG,80.08,overcast clouds,-2.0226,147.2712,
18,Tres Lagoas,BR,85.21,overcast clouds,-20.7511,-51.6783,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    Lat=row['Lat']
    Lng=row['Lng']
    
    coord=f"{Lat},{Lng}"

    params.update({"location":coord})
    
     
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("done")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
done


In [15]:
# 7. Drop the rows where there is no Hotel Name.
try:
    clean_hotel_df=hotel_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_hotel_df=hotel_df

In [16]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>{Weather Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))